# Notebook 3 – Hindsight Experience Replay (HER) et Sauvegarde Avancée

Dans ce troisième notebook, nous allons aborder plusieurs sujets avancés :

1. **Hindsight Experience Replay (HER)**, une technique permettant d'entraîner un agent sur des tâches de type "Goal-Conditioned RL" (où l’agent doit atteindre un objectif paramétrable),
2. un **exemple pratique** avec l’environnement `parking-v0` (issu de la bibliothèque [highway-env](https://github.com/eleurent/highway-env)),
3. la **sauvegarde/chargement avancés** d’un modèle, incluant la sauvegarde et le rechargement de la buffer d’expérience (replay buffer).

Cet environnement `parking-v0` est un cas classique d’apprentissage par renforcement à but : la récompense dépend d’un objectif (ici, se garer à un endroit précis, avec la bonne orientation).

Nous verrons comment utiliser HER avec différents algorithmes (SAC, DDPG), comment **sauvegarder/recharger** un modèle **et** sa mémoire de rejouage, et comment **évaluer** l’agent.


**Rappel sur le “Goal-Conditioned RL”**  
- Dans certaines tâches, un “goal” (objectif) fait partie de l’état désiré. Par exemple, la position finale d’un bras robotique, la place de parking à occuper, etc.  
- Les observations se présentent souvent comme un dictionnaire : `{'observation': ..., 'desired_goal': ..., 'achieved_goal': ...}`.  
- HER (Hindsight Experience Replay) ré-étiquette des transitions a posteriori pour rendre l’apprentissage plus efficace, surtout quand la récompense est très clairsemée (sparse).


## Installation des dépendances

Sous Windows, nous n’utilisons pas de commandes `apt-get`. Nous procédons uniquement par `pip` pour installer :

- Stable-Baselines3 (avec le `[extra]`),
- highway-env pour l’environnement `parking-v0`,
- (Optionnel) `moviepy` pour l’enregistrement de vidéos.

```bash
pip install "stable-baselines3[extra]>=2.0.0a4"
pip install highway-env
pip install moviepy
```

Dans un notebook Python, on peut faire :
```python
%pip install "stable-baselines3[extra]>=2.0.0a4" highway-env moviepy
```

In [1]:
# Installation par commande magique Notebook (Windows-friendly, pas de apt-get)
%pip install "stable-baselines3[extra]>=2.0.0a4" highway-env moviepy --quiet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Imports Essentiels

Nous importons :
- `HerReplayBuffer` : le buffer de rejouage spécialisé pour HER,
- des algorithmes (SAC, DDPG) compatibles avec HER (il faut un algo off-policy pour combiner avec HER),
- l’environnement `parking-v0` depuis highway_env,
- NumPy, etc.

In [2]:
import gymnasium as gym
import highway_env
import numpy as np

from stable_baselines3 import HerReplayBuffer, SAC, DDPG
from stable_baselines3.common.noise import NormalActionNoise
from stable_baselines3.common.evaluation import evaluate_policy

### Environnement Parking

[`parking-v0`](https://github.com/eleurent/highway-env#parking-env) est un environnement « goal-conditioned » : la position et l’orientation cibles font partie de l’`info['goal']`. Pour résoudre cette tâche, on doit apprendre à manœuvrer la voiture pour qu’elle se gare.

![parking-env](https://raw.githubusercontent.com/eleurent/highway-env/gh-media/docs/media/parking-env.gif)


### Création de l’environnement Gym

In [3]:
env = gym.make("parking-v0")
obs, _ = env.reset()
print("Observation :", obs.keys())
print("Exemple d'observation['observation']:", obs["observation"].shape)
print("Exemple d'observation['desired_goal']:", obs["desired_goal"].shape)

Observation : odict_keys(['observation', 'achieved_goal', 'desired_goal'])
Exemple d'observation['observation']: (6,)
Exemple d'observation['desired_goal']: (6,)


Par défaut, l’action est continue (2 dimensions : accélération et direction). On peut vérifier en imprimant `env.action_space` ou `env.observation_space`.

**Structure de l’observation**  
- `obs['observation']`: informations sur la voiture (position, vitesse, angle...).  
- `obs['desired_goal']`: position/angle cible (le “parking spot”).  
- `obs['achieved_goal']`: l’état effectivement atteint par la voiture.  

La récompense dépend souvent de la distance entre `achieved_goal` et `desired_goal`. HER va ré-étiqueter certains buts pour générer des transitions artificiellement “réussies”.


## Entraîner un agent SAC avec HER

La configuration de `HerReplayBuffer` est centrale ici. Nous choisissons :
- `goal_selection_strategy="future"` (la stratégie la plus courante, on va remplacer le but original par un but futur observé dans le même épisode),
- `n_sampled_goal=4` (on crée 4 transitions artificielles par transition réelle),
- des hyperparamètres un peu custom pour *SAC* : `batch_size`, `policy_kwargs`, etc.

Au final, l’entraînement dure un certain temps (on peut ajuster le `total_timesteps` en fonction de la machine).




In [4]:
model_sac = SAC(
    "MultiInputPolicy",
    env,
    replay_buffer_class=HerReplayBuffer,
    replay_buffer_kwargs=dict(
        n_sampled_goal=4,
        goal_selection_strategy="future",
    ),
    # on attend 1000 pas avant d'entraîner,
    # afin d'avoir au moins un épisode complet stocké.
    learning_starts=1000,  
    buffer_size=50000,
    batch_size=64,
    policy_kwargs=dict(net_arch=[64, 64]),
    train_freq=1,
    gradient_steps=1,
    verbose=1,
)
model_sac.learn(total_timesteps=5000, log_interval=100)



# Sauvegarde du modèle ET de la replay buffer avant suppression
model_sac.save("her_sac_parking")
model_sac.save_replay_buffer("her_sac_parking_replay_buffer")
del model_sac  # On supprime de la RAM



Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.



**Focus sur la stratégie `goal_selection_strategy=\"future\"`**  
- “future” signifie qu’on va remplacer le but initial par un but échantillonné **plus tard** dans la même trajectoire.  
- Cela favorise l’apprentissage, car beaucoup d’états futurs atteints sont convertis en “objectifs cibles”.  
- Alternatives : “final”, “episode”, “random” — à tester selon l’environnement.


## Rechargement du modèle et évaluation

Nous rechargeons ensuite le modèle, et on peut l’évaluer sur quelques épisodes :


In [5]:
from stable_baselines3.common.monitor import Monitor

# Rechargement
model_sac = SAC.load("her_sac_parking", env=env)

# Évaluation

eval_env = Monitor(env)  # Ajout du Monitor pour éviter les warnings
mean_reward, std_reward = evaluate_policy(model_sac, eval_env, n_eval_episodes=10, deterministic=True)

print(f"SAC Parking : reward moyen={mean_reward:.2f} +/- {std_reward:.2f}")



Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


SAC Parking : reward moyen=-84.81 +/- 79.35


La notion de « récompense » dans un environnement `goal-conditioned` (HER) reflète la distance à l’objectif et la réussite/échec à se garer. On peut inspecter `info.get("is_success", False)` pour savoir si l’épisode est terminé avec succès.

## Exemple avec DDPG

Nous pouvons reproduire la même idée avec un autre algorithme off-policy (DDPG). On ajoute souvent un bruit d’exploration, `NormalActionNoise` :

In [6]:
# On crée un bruit gaussien pour l’action
n_actions = env.action_space.shape[0]  # en général = 2
noise_std = 0.2
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=noise_std * np.ones(n_actions))

model_ddpg = DDPG(
    "MultiInputPolicy",
    env,
    replay_buffer_class=HerReplayBuffer,
    replay_buffer_kwargs=dict(
        n_sampled_goal=4,
        goal_selection_strategy="future",
    ),
    verbose=1,
    # On réduit la taille de la buffer
    buffer_size=50_000,
    learning_rate=1e-3,
    action_noise=action_noise,
    gamma=0.95,
    # batch_size plus petit
    batch_size=64,
    # Réseau plus léger
    policy_kwargs=dict(net_arch=[64, 64]),
    # On attend un peu avant d'entraîner
    learning_starts=1000,
    # On fait 1 step d'entraînement par step environnement
    train_freq=1,
    gradient_steps=1,
)

# On ne va pas jusqu'à 2e5 steps
# mais 5000 ou 10 000 pour une démo rapide
model_ddpg.learn(10_000)  # par exemple

# Sauvegarde
model_ddpg.save("her_ddpg_parking")
del model_ddpg


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 55.2     |
|    ep_rew_mean     | -28.6    |
|    success_rate    | 0        |
| time/              |          |
|    episodes        | 4        |
|    fps             | 185      |
|    time_elapsed    | 1        |
|    total_timesteps | 221      |
---------------------------------


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 52.8     |
|    ep_rew_mean     | -28.4    |
|    success_rate    | 0        |
| time/              |          |
|    episodes        | 8        |
|    fps             | 177      |
|    time_elapsed    | 2        |
|    total_timesteps | 422      |
---------------------------------


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 69.6     |
|    ep_rew_mean     | -36.6    |
|    success_rate    | 0        |
| time/              |          |
|    episodes        | 12       |
|    fps             | 154      |
|    time_elapsed    | 5        |
|    total_timesteps | 835      |
---------------------------------


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 66.4     |
|    ep_rew_mean     | -34      |
|    success_rate    | 0        |
| time/              |          |
|    episodes        | 16       |
|    fps             | 126      |
|    time_elapsed    | 8        |
|    total_timesteps | 1063     |
| train/             |          |
|    actor_loss      | 0.325    |
|    critic_loss     | 0.0262   |
|    learning_rate   | 0.001    |
|    n_updates       | 62       |
---------------------------------


...

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 78.4     |
|    ep_rew_mean     | -38.7    |
|    success_rate    | 0.0114   |
| time/              |          |
|    episodes        | 88       |
|    fps             | 85       |
|    time_elapsed    | 80       |
|    total_timesteps | 6901     |
| train/             |          |
|    actor_loss      | 1.47     |
|    critic_loss     | 0.0245   |
|    learning_rate   | 0.001    |
|    n_updates       | 5900     |
---------------------------------


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 82       |
|    ep_rew_mean     | -39.7    |
|    success_rate    | 0.0109   |
| time/              |          |
|    episodes        | 92       |
|    fps             | 87       |
|    time_elapsed    | 86       |
|    total_timesteps | 7543     |
| train/             |          |
|    actor_loss      | 1.78     |
|    critic_loss     | 0.0496   |
|    learning_rate   | 0.001    |
|    n_updates       | 6542     |
---------------------------------


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 89.8     |
|    ep_rew_mean     | -43.2    |
|    success_rate    | 0.0104   |
| time/              |          |
|    episodes        | 96       |
|    fps             | 89       |
|    time_elapsed    | 96       |
|    total_timesteps | 8616     |
| train/             |          |
|    actor_loss      | 1.51     |
|    critic_loss     | 0.0196   |
|    learning_rate   | 0.001    |
|    n_updates       | 7615     |
---------------------------------


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 95.5     |
|    ep_rew_mean     | -45.1    |
|    success_rate    | 0.01     |
| time/              |          |
|    episodes        | 100      |
|    fps             | 92       |
|    time_elapsed    | 103      |
|    total_timesteps | 9549     |
| train/             |          |
|    actor_loss      | 1.79     |
|    critic_loss     | 0.0161   |
|    learning_rate   | 0.001    |
|    n_updates       | 8548     |
---------------------------------


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 94.3     |
|    ep_rew_mean     | -44.5    |
|    success_rate    | 0.02     |
| time/              |          |
|    episodes        | 104      |
|    fps             | 92       |
|    time_elapsed    | 104      |
|    total_timesteps | 9648     |
| train/             |          |
|    actor_loss      | 1.61     |
|    critic_loss     | 0.0131   |
|    learning_rate   | 0.001    |
|    n_updates       | 8647     |
---------------------------------


In [7]:
model_ddpg = DDPG.load("her_ddpg_parking", env=env)
mean_reward, std_reward = evaluate_policy(model_ddpg, env, n_eval_episodes=10, deterministic=True)
print(f"DDPG Parking : reward moyen={mean_reward:.2f} +/- {std_reward:.2f}")


Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


C:\Users\jsboi\AppData\Local\Programs\Python\Python311\Lib\site-packages\stable_baselines3\common\evaluation.py:71: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


DDPG Parking : reward moyen=-93.83 +/- 96.82


# Sauvegarde et Chargement de la Replay Buffer

Une fonctionnalité avancée de Stable-Baselines3 est la **possibilité de sauvegarder aussi la buffer de rejouage** (replay buffer) :

- Par défaut, `model.save(...)` **ne** sauvegarde **pas** la replay buffer, car celle-ci peut être très volumineuse (plusieurs Go si on utilise des images, par ex.).
- Mais on peut la sauvegarder à part avec `model.save_replay_buffer(path)`, puis la recharger avec `model.load_replay_buffer(path)`.

Cela permet de **reprendre un entraînement** où on l’avait laissé, en conservant tout l’historique d’expériences collectées. Sur des environnements complexes, c’est très utile pour éviter de tout recommencer !

Exemple :

In [8]:
# 1) Sauvegarde
model_sac.save("her_sac_parking")  # ne sauvegarde pas la replay buffer
model_sac.save_replay_buffer("her_sac_parking_replay_buffer")

# 2) Rechargement du modèle + de la buffer
model_sac_2 = SAC.load("her_sac_parking", env=env)
print("Taille de la replay buffer AVANT rechargement :", model_sac_2.replay_buffer.size())

model_sac_2.load_replay_buffer("her_sac_parking_replay_buffer")
print("Taille de la replay buffer APRÈS rechargement :", model_sac_2.replay_buffer.size())


Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Taille de la replay buffer AVANT rechargement : 0
Taille de la replay buffer APRÈS rechargement : 0



**Reprendre l’entraînement**  
- Après avoir chargé le modèle et la replay buffer, on peut appeler `model_sac_2.learn(N)` pour continuer exactement là où l’on s’était arrêté.  
- Utile pour *checkpoint* l’entraînement de temps en temps, sans perdre l’historique des transitions passées (particulièrement en off-policy).


# Enregistrement vidéo sous Windows

Comme évoqué dans les notebooks précédents, sous Windows, pas besoin de démarrer un display virtuel via `xvfb`. On peut simplement enregistrer en mode `rgb_array`. 

Voici une fonction utilitaire pour enregistrer une vidéo d’un agent (la même que dans les notebooks précédents, adaptée pour Windows) :

In [9]:
import base64
from pathlib import Path
from IPython.display import HTML

from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv

def record_video(env_id, model, video_length=1000, prefix="", video_folder="videos/"):
    # On crée un DummyVecEnv pour enregistrer
    eval_env = DummyVecEnv([lambda: gym.make(env_id, render_mode="rgb_array")])
    
    # On active le recorder vidéo :
    eval_env = VecVideoRecorder(
        eval_env,
        video_folder,
        record_video_trigger=lambda step: step == 0,
        video_length=video_length,
        name_prefix=prefix,
    )

    obs = eval_env.reset()
    for _ in range(video_length):
        action, _ = model.predict(obs, deterministic=True)
        obs, _, done, info = eval_env.step(action)

    eval_env.close()

def show_videos(video_path="videos", prefix=""):
    """
    Affiche toutes les vidéos mp4 dans le dossier spécifié.
    """
    mp4_list = list(Path(video_path).glob(f"{prefix}*.mp4"))
    if len(mp4_list) == 0:
        print("Aucune vidéo trouvée.")
        return

    html_video = ""
    for mp4 in mp4_list:
        video_b64 = base64.b64encode(mp4.read_bytes()).decode("ascii")
        html_video += f"<video alt='{mp4}' autoplay loop controls style='height: 400px;'>\n" \
                      f"<source src='data:video/mp4;base64,{video_b64}' type='video/mp4' />\n" \
                      "</video>\n"
    return HTML(html_video)

Pour tester, nous pouvons enregistrer une vidéo de `model_sac` ou `model_ddpg`. (Attention : `parking-v0` peut boucler un peu longtemps si on met un `video_length` trop grand.)

In [10]:
record_video("parking-v0", model_sac, video_length=500, prefix="sac-parking")
show_videos("videos", prefix="sac-parking")

Saving video to C:\dev\CoursIA\videos\sac-parking-step-0-to-step-500.mp4
MoviePy - Building video C:\dev\CoursIA\videos\sac-parking-step-0-to-step-500.mp4.
MoviePy - Writing video C:\dev\CoursIA\videos\sac-parking-step-0-to-step-500.mp4



frame_index:   0%|          | 0/501 [00:00<?, ?it/s, now=None]

frame_index:  27%|██▋       | 134/501 [00:00<00:00, 1322.38it/s, now=None]

frame_index:  62%|██████▏   | 313/501 [00:00<00:00, 1593.46it/s, now=None]

frame_index:  95%|█████████▍| 475/501 [00:00<00:00, 1575.83it/s, now=None]

MoviePy - Done !
MoviePy - video ready C:\dev\CoursIA\videos\sac-parking-step-0-to-step-500.mp4


Vous devriez voir la voiture tenter de se garer…

## Conclusion

Dans ce troisième notebook, nous avons abordé des fonctionnalités avancées de Stable-Baselines3 :

- **Hindsight Experience Replay (HER)**, qui permet d’apprendre efficacement sur des tâches à but (objectif) en ré-étiquetant des transitions passées,
- l’utilisation de **SAC** ou **DDPG** avec HER (algorithmes off-policy),
- la **sauvegarde et le rechargement** de la replay buffer pour reprendre un entraînement ultérieurement,
- l’enregistrement vidéo « friendly pour Windows », sans dépendances `apt-get`.

Avec cela, vous disposez d’une base solide pour traiter des tâches plus complexes en Apprentissage par Renforcement, où l’agent doit atteindre des objectifs spécifiques.